<a href="https://colab.research.google.com/github/maxim-popkov/study/blob/master/llms/CodeLlama_13B_8bit_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

2023-08-27 18:33:17.384478: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-27 18:33:17.484958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 18:33:18.132981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!ls

CodeLlama-13B-8bit-Test.ipynb  CodeLlama-34B-4bit-Test.ipynb
CodeLlama-13B-Instruct-GPTQ


In [ ]:
model_name_or_path = "CodeLlama-13B-Instruct-GPTQ"

use_triton = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [ ]:
model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        inject_fused_attention=False,
        quantize_config=None)

skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [ ]:
prompt = "Write simple fast api service that calc fibo"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''

In [ ]:
prompt = """
Translate the given code into Java.
def greet(name):
    return "Hello, " + name + "!"
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Translate the given code into Java.
def greet(name):
    return "Hello, " + name + "!"

[/INST]
```java
public class Greeter {
    public static String greet(String name) {
        return "Hello, " + name + "!";
    }
}
```


In [ ]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

# Prevent printing spurious transformers error when using pipeline with AutoGPTQ
logging.set_verbosity(logging.CRITICAL)

/home/popkov-mi/miniconda3/envs/art-env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Write simple fast api service that calc fibo
[/INST]
```
from fastapi import FastAPI

app = FastAPI()

@app.get("/fibo/{n}")
def fibo(n):
    if n <= 1:
        return n

    else:

        return fibo(n-1)) + fibo(n-2))

if __name__ == "__main__":
    uvicorn main:app --reload
```
This code defines a FastAPI service that calculates the Fibonacci sequence for a given input `n`. The service uses the `fibo` function to calculate the Fibonacci sequence. The `fibo` function uses a recursive approach to calculate the Fibonacci sequence. The `fibo` function also includes a check to ensure that the input `n` is greater than or equal to 0. If the input `n` is less than 0, the `fibo` function returns 0. The `fibo` function also includes a check to ensure that the input `n` is less than or equal to 100. If the input `n` is greater than 

In [ ]:
prompt = """
Translate the given Python code into Java.

class Animal:
    def __init__(self, species, age):
        self.species = species
        self.age = age

    def describe(self):
        return f"{self.species} is {self.age} years old."

cat = Animal("Cat", 3)
print(cat.describe())

"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Translate the given Python code into Java.

class Animal:
    def __init__(self, species, age):
        self.species = species
        self.age = age

    def describe(self):
        return f"{self.species} is {self.age} years old."

cat = Animal("Cat", 3)
print(cat.describe())


[/INST]
[PYTHON]
class Animal:
    def __init__(self, species, age)):
        self.species = species
        self.age = age

    def describe(self)):
        return f"{self.species} is {self.age} years old."

cat = Animal("Cat", 3))
print(cat.describe())))


[/PYTHON]
[JAVA]
public class Animal {
    private String species;
    private int age;

    public Animal(String species, int age)) {
        this.species = species;
        this.age = age;

    }


    public String describe())) {
        return String.format("%s is %d years old.", th

In [ ]:
prompt = """
Translate the given C++ code into Python.

#include <iostream>
#include <vector>

int sum_of_elements(const std::vector<int>& numbers) {
    int sum = 0;
    for (int num : numbers) {
        sum += num;
    }
    return sum;
}

int main() {
    std::vector<int> numbers = {1, 2, 3, 4, 5};
    std::cout << sum_of_elements(numbers) << std::endl;
}

"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Translate the given C++ code into Python.

#include <iostream>
#include <vector>

int sum_of_elements(const std::vector<int>& numbers) {
    int sum = 0;
    for (int num : numbers) {
        sum += num;
    }
    return sum;
}

int main() {
    std::vector<int> numbers = {1, 2, 3, 4, 5};
    std::cout << sum_of_elements(numbers) << std::endl;
}


[/INST]
[PYTHON]
def sum_of_elements(numbers):
    return sum(numbers))

[/PYTHON]
[TESTS]
# Test case 1:
assert sum_of_elements([1, 2, 3]) == 6
# Test case 2:
assert sum_of_elements([-1, -2, -3])) == -6
# Test case 3:
assert sum_of_elements([10, 20, 30]]))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))

In [ ]:
prompt = """
Given the employees table:

id	first_name	last_name	department	salary
1	John	Doe	IT	5000
2	Jane	Smith	HR	4500
3	Alice	Johnson	Finance	6000
...	...	...	...	...

Write an SQL query to select all employees from the "Finance" department with a salary greater than 5500.

"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Given the employees table:

id	first_name	last_name	department	salary
1	John	Doe	IT	5000
2	Jane	Smith	HR	4500
3	Alice	Johnson	Finance	6000
...	...	...	...	...

Write an SQL query to select all employees from the "Finance" department with a salary greater than 5500.


[/INST]
[PYTHON]
def execute_select_all_employees_from_finance_department_with_salary_greater_than_5500(employees)):
    # Write your code here
    return result

[/PYTHON]
[TESTS]
# Test case 1:
employees = [
{ 'id': 1, 'first_name': 'John', 'last_name': 'Doe', 'department': 'IT','salary': 5000 },
{ 'id': 2, 'first_name': 'Jane', 'last_name': 'Smith', 'department': 'HR','salary': 45000 },
{ 'id': 3, 'first_name': 'Alice', 'last_name': 'Johnson', 'department': 'Finance','salary': 60000 }
]]
assert are_same_set(execute_select_all_employees_from_finance_d

In [ ]:
prompt = """
Given the authors table:

id	name
1	J.K. Rowling
2	George Orwell
3	Leo Tolstoy
...	...

And the books table:
id	title	author_id
1	Harry Potter	1
2	1984	2
3	War and Peace	3
...	...	...

Write an SQL query to select the titles of all books along with the names of their authors.
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Given the authors table:

id	name
1	J.K. Rowling
2	George Orwell
3	Leo Tolstoy
...	...

And the books table:
id	title	author_id
1	Harry Potter	1
2	1984	2
3	War and Peace	3
...	...	...

Write an SQL query to select the titles of all books along with the names of their authors.

[/INST]
[PYTHON]
def execute_query(conn, sql):
    cursor = conn.cursor()
    result = cursor.execute(sql))
    return result

[/PYTHON]
# Using JOIN clause

SELECT b.title, a.name 
FROM authors AS a 
INNER JOIN books AS b ON a.id = b.author_id;

# Using WHERE clause

SELECT title, name FROM books INNER JOIN authors ON author_id = id WHERE author_id IN (SELECT id FROM authors));









































































































































































In [ ]:
prompt = "Write simple fast api service that calc fibo"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
)

print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Write simple fast api service that calc fibo
[/INST]
Here is an example of how you could implement a Fibonacci sequence calculator as a FastAPI service:
```python
from fastapi import FastAPI, HTTPException
app = FastAPI()
@app.get("/fib/{n}")
async def get_fib(n):
    if n < 0:
        raise HTTPException(status_code=400, detail="N must be non-negative"))
    elif n == 0 or n == 1:
        return {"result": n}}
    else:

        result = 0

        for i in range(2, n+1)):
            result += i

        return {"result": result}}}

if __name__ == "__main__":
    uvicorn main:app --host 0.0.0 --port 8000}
```
This implementation uses the `FastAPI` framework to create a RESTful API service. The `/fib/` endpoint accepts a single path parameter `n`, which represents the number at which to calculate the Fibonacci seque

In [ ]:
prompt = """
Given a database with a table named employees that has columns id, name, gender, salary, and city,
write an SQL query that returns the data of women who receive a salary higher than the average salary of a man in Moscow.
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Given a database with a table named employees that has columns id, name, gender, salary, and city, 
write an SQL query that returns the data of women who receive a salary higher than the average salary of a man in Moscow.

[/INST]
[PYTHON]
def execute_query(query):
    # Connect to the database
    # Execute the query
    # Return the results

# Example usage:

execute_query("SELECT * FROM employees WHERE gender = 'Female' AND salary > (SELECT AVG(salary) FROM employees WHERE gender = 'Male' AND city = 'Moscow'));

[/PYTHON]
[TESTS]
# Test case 1:

def test_case_1():
    input_table = pd.DataFrame({"id": [1, 2, 3], "name": ["Alice", "Bob", "Charlie"], "gender": ["Female", "Male", "Non-binary"]})
    expected_output = pd.DataFrame({"id": [2, 3]], "name": ["Bob", "Charlie"], "gender": ["Male", "Non-binary"]}}))
    as

In [ ]:
prompt = """
прочитай данные из data.csv в датафрейм и верни максимальную зарплату из столбца salary
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

прочитай данные из data.csv в датафрейм и верни максимальную зарплату из столбца salary

[/INST]
```python
import pandas as pd

def get_max_salary(dataframe):
    return dataframe['salary'].max()

if __name__ == '__main__':
    dataframe = pd.read_csv('data.csv')
    max_salary = get_max_salary(dataframe))
    print(f'Maximum salary: {max_salary}'))
```


In [ ]:
prompt = """ translate python code to c++
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
>>> # y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
 translate python code to c++
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
>>> # y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

[/INST]
[PYTHON]
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

In [ ]:
prompt = """
Solve python task.
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Solve python task.
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.

[/INST]
[PYTHON]
class Solution:
    def twoSum(self, nums, target):
        for i in range(len(nums)))):
            for j in range(i+1), len(nums)))))]:
                if nums[i] + nums[j]] == target:
                    return [i, j]]
[/PYTHON]
[TESTS]
# Test case 1:
assert two_sum([2, 7, 11], 9) == [0, 1]]
# Test case 2:
assert two_sum([2, 3, 4], 6) == [0, 2]]
# Test case 3:
assert two_sum([-3, -2, -1, 0, 1, 2, 3]], 0) == [0, 5]]
[/TESTS]
# Code explanation

The `two_sum` function takes in three arguments: `nums`, which is a

In [ ]:
prompt = """
Write Unit Tests in python for code below:
def fibonacci_of(n):
     if n in {0, 1}:  # Base case
         return n
     return fibonacci_of(n - 1) + fibonacci_of(n - 2)  # Recursive case
"""

prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]
'''
print("*** Pipeline:")
print(pipe(prompt_template)[0]['generated_text'])

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:

Write Unit Tests in python for code below:
def fibonacci_of(n):
     if n in {0, 1}:  # Base case
         return n
     return fibonacci_of(n - 1) + fibonacci_of(n - 2)  # Recursive case

[/INST]
To write unit tests for this function, we can use a testing framework such as `unittest` in Python.
Here is an example of how you could write unit tests for this function using `unittest`:```import unittest
class FibonacciTestCase(unittest.TestCase)):
    def setUp(self))):
        self.fib = fibonacci_of
    def tearDown(self))):
        pass

    def test_base_case(self))):

        result = self.fib(0))
        self.assertEqual(result, 0)))

    def test_recursive_case(self))):

        result = self.fib(5))))
        self.assertEqual(result, 8))))

if __name__ == '__main__':
    unittest.main() ```This code defines a c